In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv('data/ab_data.zip')

In [5]:
df

,user_id,timestamp,group,converted
0,851104,2017-01-21,A,0
1,804228,2017-01-12,A,0
2,661590,2017-01-11,B,0
3,853541,2017-01-08,B,0
4,864975,2017-01-21,A,1
...,...,...,...,...
290477,751197,2017-01-03,A,0
290478,945152,2017-01-12,A,0
290479,734608,2017-01-22,A,0
290480,697314,2017-01-15,A,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290482 entries, 0 to 290481
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    290482 non-null  int64 
 1   timestamp  290482 non-null  object
 2   group      290482 non-null  object
 3   converted  290482 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 8.9+ MB


In [7]:
# Конвертирую datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290482 entries, 0 to 290481
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   user_id    290482 non-null  int64         
 1   timestamp  290482 non-null  datetime64[ns]
 2   group      290482 non-null  object        
 3   converted  290482 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 8.9+ MB


In [9]:
a_df = df[df['group'] == 'A']

In [10]:
a_df

,user_id,timestamp,group,converted
0,851104,2017-01-21,A,0
1,804228,2017-01-12,A,0
4,864975,2017-01-21,A,1
5,936923,2017-01-10,A,0
7,719014,2017-01-17,A,0
...,...,...,...,...
290475,718310,2017-01-21,A,0
290477,751197,2017-01-03,A,0
290478,945152,2017-01-12,A,0
290479,734608,2017-01-22,A,0


In [11]:
b_df = df[df['group'] == 'B']

In [12]:
b_df

,user_id,timestamp,group,converted
2,661590,2017-01-11,B,0
3,853541,2017-01-08,B,0
6,679687,2017-01-19,B,1
8,817355,2017-01-04,B,1
9,839785,2017-01-15,B,1
...,...,...,...,...
290466,677163,2017-01-03,B,0
290469,925675,2017-01-07,B,0
290472,643562,2017-01-02,B,0
290476,822004,2017-01-04,B,0


In [14]:
df.groupby(['timestamp', 'group']).agg({'user_id': 'count', 'converted': 'sum'}).reset_index()

,timestamp,group,user_id,converted
0,2017-01-02,A,2851,357
1,2017-01-02,B,2848,339
2,2017-01-03,A,6584,749
3,2017-01-03,B,6620,754
4,2017-01-04,A,6573,797
5,2017-01-04,B,6538,763
6,2017-01-05,A,6419,791
7,2017-01-05,B,6509,743
8,2017-01-06,A,6604,762
9,2017-01-06,B,6735,834
